# **PREDICCIONES PARA TODA LA POBLACION SIN AGREGACION**

In [115]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sbn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import random as random
import plotly.express as px
import tensorflow as tf
from tensorflow.keras import layers

def set_output_precision(decimals):
  """
  format the output of the all the data structures
  with an specific number of decimals
  """
  np.set_printoptions(precision=decimals)
  into='{'+':.{}f'.format(decimals)+'}'
  pd.options.display.float_format = into.format
  torch.set_printoptions(precision=decimals)
  pass

def plot_ts(df,dfx="Minute",dfy="METS",_title="DF minute x Mets"):
  if not isinstance(df,pd.DataFrame):
    df = pd.DataFrame({'METS': df, 'Minute': range(len(df))})

  plt.figure()
  fig = px.line(df, x = dfx, y = dfy , title = _title)
  fig.update_xaxes(
      rangeslider_visible = True,
      rangeselector = dict(
          buttons = list([
              dict(count=1,label="1y",step="year",stepmode="backward"),
              dict(count=2,label="2y",step="year",stepmode="backward"),
              dict(count=3,label="3y",step="year",stepmode="backward"),
              dict(step="all")
          ])
      )

  )
  fig.show()

def plot_predictions_vs_real(predictions, reals):
  df = pd.DataFrame()
  number_of_points = len(predictions)
  df["time"] = range(0,number_of_points)
  df["participant"] = "prediction"
  df["value"] = predictions
  for i in range(0,number_of_points):
    df.loc[number_of_points+i] = [i,"real",reals[i]]

  plt.figure(1)
  fig = px.line(df, x = "time", y = "value" , title = "predictions vs reals" , color = "participant")
  fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(count=2,label="2y",step="year",stepmode="backward"),
                dict(count=3,label="3y",step="year",stepmode="backward"),
                dict(step="all")
            ])
        )

    )
  fig.show()

In [116]:
READ_LOCAL_DATA = True
COMPUTED_OPTION = 0
TEST_SIZE = 0.2
VALIDATION_SIZE = 0.15
SAVE_RESULTS = True
LOW_DATA = True

In [117]:
if READ_LOCAL_DATA:
  PATH = "Resources/Individual/"
else:
    #  We start by getting access to the drive
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/TFG/Resources/Individual/"

if LOW_DATA:
    PATH += "LowData/"

In [118]:
import gzip
documents = ['minuteY','hourY','dayY']
# with open(PATH+"minuteX"+".pkl", 'rb') as file:
#     dataX = np.array(pickle.load(file),np.float32)
file = PATH+"minuteX.pkl.gz"
dataX = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

file = PATH+documents[COMPUTED_OPTION]+".pkl.gz"
dataY = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

NUMBER_OF_PARTICIPANTS = dataX.shape[0]
print('\033[1m' + "\t SHAPES" + '\033[0m')
print(dataX.shape)
print(dataY.shape)

	 SHAPES
(25, 36355, 1440)
(25, 36355, 1440)


In [119]:
if COMPUTED_OPTION == 0:
    dataX_nuevo = dataX
    JUMP = 1
elif COMPUTED_OPTION == 1:
    JUMP = 60
    dataX_nuevo = np.ones(shape=(NUMBER_OF_PARTICIPANTS,dataX.shape[1],24))
    for k in range(0,NUMBER_OF_PARTICIPANTS):
        for i in range(0,dataX.shape[1]):
            for j in range(0,24):
                dataX_nuevo[k,i,j] = np.sum(dataX[k,i,60*j:60*(j+1)])
else:
    JUMP = 1440
    dataX_nuevo = np.ones(shape=(dataX.shape[0],1))
    for i in range(0,dataX.shape[0]):
        dataX_nuevo[i,:] = np.sum(dataX[i,:])

print('\033[1m' + "\t SHAPES" + '\033[0m')
print(dataX_nuevo.shape)
print(dataY.shape)

	 SHAPES
(25, 36355, 1440)
(25, 36355, 1440)


In [120]:
dataX_nuevo = dataX_nuevo.transpose(1,0,2)
dataY = dataY.transpose(1,0,2)
print(dataX_nuevo.shape)
print(dataY.shape)

(36355, 25, 1440)
(36355, 25, 1440)


In [121]:
#We split a test set for testing
#We split a test set for testing
# train_test_split(dataX, dataY, test_size=TEST_SIZE)
LAST_TWO_DAYS_INDEX = -1440*2
X_train, X_test, y_train, y_test = dataX_nuevo[:LAST_TWO_DAYS_INDEX,:,:],dataX_nuevo[LAST_TWO_DAYS_INDEX::JUMP,:,:],\
    dataY[:LAST_TWO_DAYS_INDEX:,:,0:1],dataY[LAST_TWO_DAYS_INDEX::JUMP,:,0:1]

print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)
VALIDATION_INDEX = int(len(X_train)*VALIDATION_SIZE)
print("Examples for validation: \n", VALIDATION_INDEX)

Examples for training
 X: (33475, 25, 1440) y: (33475, 25, 1)
Examples for test
 X: (2880, 25, 1440) y: (2880, 25, 1)
Examples for validation: 
 5021


In [122]:
tf.random.set_seed(42)
# Setup dataset hyperparameters
HORIZON = y_test.shape[2]
WINDOW_SIZE = X_train.shape[2]

# Let's build an LSTM model with the Functional API
inputs = layers.Input(shape=(NUMBER_OF_PARTICIPANTS ,WINDOW_SIZE))
# x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs) # expand input dimension to be compatible with LSTM
# print(x.shape)
# x = layers.LSTM(128, activation="relu", return_sequences=True)(x) # this layer will error if the inputs are not the right shape
x = layers.LSTM(128,return_sequences=True, activation="relu")(inputs) # using the tanh loss function results in a massive error
# print(x.shape)
# Add another optional dense layer (you could add more of these to see if they improve model performance)
# x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(HORIZON)(x)
model_LSTM = tf.keras.Model(inputs=inputs, outputs=output, name="model_5_lstm")

model_LSTM.summary()

Model: "model_5_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 25, 1440)]        0         
                                                                 
 lstm_3 (LSTM)               (None, 25, 128)           803328    
                                                                 
 dense_3 (Dense)             (None, 25, 1)             129       
                                                                 
Total params: 803,457
Trainable params: 803,457
Non-trainable params: 0
_________________________________________________________________


In [123]:
# Compile model
model_LSTM.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(),
             metrics=["mae"])

In [124]:
# Seems when saving the model several warnings are appearing: https://github.com/tensorflow/tensorflow/issues/47554
model_LSTM.fit(X_train[:-VALIDATION_INDEX],
            y_train[:-VALIDATION_INDEX],
            epochs=3,
            verbose=1,
            batch_size=128,
            validation_data=(X_train[-VALIDATION_INDEX:], y_train[-VALIDATION_INDEX:]))

Epoch 1/3
223/223 [==============================] - 28s 120ms/step - loss: 7.9260 - mae: 7.9260 - val_loss: 1.7685 - val_mae: 1.7685
Epoch 2/3
223/223 [==============================] - 27s 119ms/step - loss: 1.1394 - mae: 1.1394 - val_loss: 1.1544 - val_mae: 1.1544
Epoch 3/3
223/223 [==============================] - 25s 113ms/step - loss: 0.9210 - mae: 0.9210 - val_loss: 0.8926 - val_mae: 0.8926


In [125]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model 
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data,verbose=2)
  return tf.squeeze(forecast) # return 1D array of predictions

predictions = make_preds(model_LSTM, X_test)

90/90 [==============================] - 3s 33ms/step


In [126]:
if SAVE_RESULTS:
    file_path = 'Resources/Resultados/Matrix/'

    if LOW_DATA:
        file_path  += "LowData/"

    documents = ['minuteY','hourY','dayY']
    file = file_path+documents[COMPUTED_OPTION]+"-predictions"+".pkl.gz"
    pickle.dump(predictions, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-test"+".pkl.gz"
    pickle.dump(y_test, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-X"+".pkl.gz"
    pickle.dump(X_test, gzip.open(file, 'wb'))

In [127]:
day_before = np.array(X_test[0:1,:,:],dtype=np.float32)
predicted_values_day_one = np.ones(shape=(WINDOW_SIZE,NUMBER_OF_PARTICIPANTS,1),dtype=np.float32)
print(predicted_values_day_one.shape)
print(day_before.shape)
for i in range(0,WINDOW_SIZE):
    prediction = make_preds(model_LSTM, day_before)
    day_before[:,:,0:-1] = day_before[:,:,1:]
    day_before[:,:,-1] = prediction[:]
    predicted_values_day_one[i,:,0] = prediction[:]

(1440, 25, 1)
(1, 25, 1440)
1/1 [==============================] - 0s 20ms/step


In [128]:
SECOND_DAY = X_test.shape[2]
day_before = np.array(X_test[SECOND_DAY:SECOND_DAY+1,:,:],dtype=np.float32)
predicted_values_day_two = np.ones(shape=(WINDOW_SIZE,NUMBER_OF_PARTICIPANTS,1),dtype=np.float32)
for i in range(0,WINDOW_SIZE):
    prediction = make_preds(model_LSTM, day_before)
    day_before[:,:,0:-1] = day_before[:,:,1:]
    day_before[:,:,-1] = prediction[:]
    predicted_values_day_two[i,:,0] = prediction[:]

1/1 [==============================] - ETA: 0s

In [132]:
poblational_predicted_values_day_one = np.sum(predicted_values_day_one,axis=1)
poblational_predicted_values_day_two = np.sum(predicted_values_day_two,axis=1)
poblational_y_test = np.sum(y_test,axis=1)

In [133]:
print("Predecidos : " + str(int(np.sum(poblational_predicted_values_day_one[:,0]))))
print("Reales : " + str(int(np.sum(poblational_y_test[0:WINDOW_SIZE,0]))))
if COMPUTED_OPTION < 2:
    plot_predictions_vs_real(poblational_predicted_values_day_one[:,0],poblational_y_test[0:WINDOW_SIZE,0])

Predecidos : -59809832960
Reales : 57047


<Figure size 640x480 with 0 Axes>

In [144]:
print("Predecidos : " + str(int(np.sum(poblational_predicted_values_day_two[:,0]))))
print("Reales : " + str(int(np.sum(poblational_y_test[SECOND_DAY:,0]))))
if COMPUTED_OPTION < 2:
    plot_predictions_vs_real(poblational_predicted_values_day_two[:100,0],poblational_y_test[SECOND_DAY:SECOND_DAY+100,0])

Predecidos : -120130838528
Reales : 55143


<Figure size 640x480 with 0 Axes>